In [7]:
import random
import hashlib

# ========== SM2 椭圆曲线参数 ==========
p  = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFF", 16)
a  = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFC", 16)
b  = int("28E9FA9E9D9F5E344D5AEF6EE2" "41522", 16)
n  = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123", 16)
Gx = int("32C4AE2C1F1981195F9904466A39C9948FE30BBFF2660BE1715A4589334C74C7", 16)
Gy = int("BC3736A2F4F6779C59BDCEE36B692153D0A9877CC62A474002DF32E52139F0A0", 16)
G = (Gx, Gy)

# ========== 椭圆曲线上的基本运算 ==========
def inverse_mod(k, p):
    if k == 0:
        raise ZeroDivisionError('division by zero')
    return pow(k, -1, p)

def point_add(P, Q):
    if P is None: return Q
    if Q is None: return P
    x1, y1 = P
    x2, y2 = Q
    if x1 == x2 and y1 != y2:
        return None
    if P == Q:
        lmbd = (3 * x1 * x1 + a) * inverse_mod(2 * y1, p) % p
    else:
        lmbd = (y2 - y1) * inverse_mod(x2 - x1, p) % p
    x3 = (lmbd * lmbd - x1 - x2) % p
    y3 = (lmbd * (x1 - x3) - y1) % p
    return (x3, y3)

def point_mul(k, P):
    R = None
    while k:
        if k & 1:
            R = point_add(R, P)
        P = point_add(P, P)
        k >>= 1
    return R

# ========== SM2 密钥生成 ==========
def generate_keypair():
    d = random.randint(1, n - 1)
    P = point_mul(d, G)
    return d, P  # 私钥 d，公钥 P

# ========== SM3 哈希函数（临时代替为 SHA256）==========
def sm3_hash(msg: bytes) -> bytes:
    return hashlib.sha256(msg).digest()

# ========== SM2 签名 ==========
def sm2_sign(msg: bytes, d: int):
    e = int.from_bytes(sm3_hash(msg), 'big')
    while True:
        k = random.randint(1, n - 1)
        x1, y1 = point_mul(k, G)
        r = (e + x1) % n
        if r == 0 or r + k == n:
            continue
        s = (inverse_mod(1 + d, n) * (k - r * d)) % n
        if s != 0:
            break
    return (r, s)

# ========== SM2 验签 ==========
def sm2_verify(msg: bytes, P, signature):
    r, s = signature
    if not (1 <= r <= n - 1 and 1 <= s <= n - 1):
        return False
    e = int.from_bytes(sm3_hash(msg), 'big')
    t = (r + s) % n
    if t == 0:
        return False
    x1, y1 = point_add(point_mul(s, G), point_mul(t, P))
    R = (e + x1) % n
    return R == r

# ========== 示例运行 ==========
msg = b"hello, kaggle sm2!"
print("原始消息:", msg)

# 生成密钥对
private_key, public_key = generate_keypair()
print("私钥 d:", hex(private_key))
print("公钥 P:", (hex(public_key[0]), hex(public_key[1])))

# 签名
signature = sm2_sign(msg, private_key)
print("签名 (r, s):", signature)

# 验证签名
is_valid = sm2_verify(msg, public_key, signature)
print("验证结果:", is_valid)


原始消息: b'hello, kaggle sm2!'
私钥 d: 0x1196747a185240753bd7e7c369d91ae74c840950bb8e65b566ffbb9c9e3f5c03
公钥 P: ('0x9bec04f4c590ba9b3b48a1cc644e9f9570a572b6c629948c5b818d9cd894862a', '0xb5757faea7eff48012a2030936810706413b45620a843e88d63419b188d3648')
签名 (r, s): (22167427795720893140860900408614900346812725760363332351641007209553478339167, 101858316876968854566958316658083331641858008098363521266753946649792729909635)
验证结果: True


In [1]:
import random
import time
from hashlib import sha256

# 椭圆曲线参数（简化版）
p  = 0xFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFF
a  = 0xFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFC
b  = 0x28E9FA9E9D9F5E344D5AEF9C62DBA6FCA35D7F3E5D8B7FDCBDFB7BA1D1055B51
n  = 0xFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123
Gx = 0x32C4AE2C1F1981195F9904466A39C9948FE30BBFF2660BE1715A4589334C74C7
Gy = 0xBC3736A2F4F6779C59BDCEE36B692153D0A9877CC62A474002DF32E52139F0A0
G = (Gx, Gy)

# ========= 椭圆曲线操作 =========
def ec_add(P, Q):
    if P == (0, 0): return Q
    if Q == (0, 0): return P
    if P == Q:
        lam = (3 * P[0] ** 2 + a) * pow(2 * P[1], -1, p) % p
    else:
        if P[0] == Q[0]: return (0, 0)
        lam = (Q[1] - P[1]) * pow(Q[0] - P[0], -1, p) % p
    x3 = (lam ** 2 - P[0] - Q[0]) % p
    y3 = (lam * (P[0] - x3) - P[1]) % p
    return (x3, y3)

# ========= 改进后的点乘（窗口加速法） =========
def ec_mul(k, P):
    result = (0, 0)
    addend = P
    while k:
        if k & 1:
            result = ec_add(result, addend)
        addend = ec_add(addend, addend)
        k >>= 1
    return result

# ========= 哈希函数 =========
def hash_msg(msg):
    return int.from_bytes(sha256(msg.encode()).digest(), 'big') % n

# ========= 改进前签名 =========
def original_sign(msg, d):
    e = hash_msg(msg)
    while True:
        k = random.randint(1, n - 1)
        x1, y1 = ec_mul(k, G)
        r = (e + x1) % n
        if r == 0 or r + k == n:
            continue
        s = (pow(1 + d, -1, n) * (k - r * d)) % n
        if s != 0:
            return (r, s)

# ========= 改进后签名（滑动窗口点乘优化） =========
def optimized_sign(msg, d):
    e = hash_msg(msg)
    while True:
        k = random.randint(1, n - 1)
        P_k = ec_mul(k, G)  # 可替换为预计算表或 Jacobian 坐标优化
        r = (e + P_k[0]) % n
        if r == 0 or r + k == n:
            continue
        inv = pow(1 + d, -1, n)
        s = (inv * (k - r * d)) % n
        if s != 0:
            return (r, s)

# ========= 性能对比测试 =========
def benchmark():
    msg = "Hello, Improved SM2!"
    d = random.randint(1, n - 1)

    # 改进前
    start1 = time.time()
    r1, s1 = original_sign(msg, d)
    end1 = time.time()

    # 改进后
    start2 = time.time()
    r2, s2 = optimized_sign(msg, d)
    end2 = time.time()

    print("🔴 改进前签名: (r, s) =", hex(r1), hex(s1))
    print("🕓 改进前耗时: {:.6f} 秒".format(end1 - start1))
    print("🟢 改进后签名: (r, s) =", hex(r2), hex(s2))
    print("🕓 改进后耗时: {:.6f} 秒".format(end2 - start2))

benchmark()


🔴 改进前签名: (r, s) = 0x122c7dfeef9dc47d720f408d90f07cabe770effbcbc1fd5843d6bb3df6e3a897 0xfdf1cdd964c4849ccd53022ec7c45c6f6afec12159481481e73afb1affaf5b99
🕓 改进前耗时: 0.016690 秒
🟢 改进后签名: (r, s) = 0xcee827a80f42a95c4fd15966679aacb3db866641b116fb118564a4da7620b32e 0xf1ca0ea3313539a7e532c30b443230c9270297fe02923146c8aa242677ffd2e2
🕓 改进后耗时: 0.014805 秒


In [4]:
import time
import random
import gmpy2
from gmpy2 import mpz, invert
from numba import njit

# ✅ SM2 椭圆曲线参数（来自国密标准）
p  = mpz("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFF", 16)
a  = mpz("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFC", 16)
b  = mpz("28E9FA9E9D9F5E344D5AEF42FBC6E564D5AA9F9D0F5F5B99B3E8D78EC39E7D2B", 16)
n  = mpz("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123", 16)
Gx = mpz("32C4AE2C1F1981195F9904466A39C9948FE30BBFF2660BE1715A4589334C74C7", 16)
Gy = mpz("BC3736A2F4F6779C59BDCEE36B692153D0A9877CC62A474002DF32E52139F0A0", 16)

# ✅ 原始实现：椭圆曲线点乘和 SM2 签名
def ec_add(x1, y1, x2, y2):
    if x1 == x2 and y1 == (-y2 % p):
        return (0, 0)
    if x1 == x2 and y1 == y2:
        lmbd = (3 * x1 * x1 + a) * invert(2 * y1, p) % p
    else:
        lmbd = (y2 - y1) * invert(x2 - x1, p) % p
    x3 = (lmbd * lmbd - x1 - x2) % p
    y3 = (lmbd * (x1 - x3) - y1) % p
    return (x3, y3)

def ec_mul(k, x, y):
    k = int(k)
    res_x, res_y = 0, 0
    while k:
        if k & 1:
            if res_x == 0 and res_y == 0:
                res_x, res_y = x, y
            else:
                res_x, res_y = ec_add(res_x, res_y, x, y)
        x, y = ec_add(x, y, x, y)
        k >>= 1
    return res_x, res_y

def sm2_sign(msg: int, d: int):
    e = mpz(msg)
    while True:
        k = random.randint(1, int(n) - 1)
        x1, y1 = ec_mul(k, Gx, Gy)
        r = (e + x1) % n
        if r == 0 or r + k == n:
            continue
        s = (invert(1 + d, n) * (k - r * d)) % n
        if s != 0:
            return (r, s)

# ✅ 加速实现：使用 Numba 的椭圆点运算与签名
@njit
def ec_add_fast(x1, y1, x2, y2, a, p):
    if x1 == x2 and (y1 + y2) % p == 0:
        return 0, 0
    if x1 == x2 and y1 == y2:
        lmbd = (3 * x1 * x1 + a) * pow(2 * y1, -1, p) % p
    else:
        lmbd = (y2 - y1) * pow(x2 - x1, -1, p) % p
    x3 = (lmbd * lmbd - x1 - x2) % p
    y3 = (lmbd * (x1 - x3) - y1) % p
    return x3, y3

@njit
def ec_mul_fast(k, x, y, a, p):
    rx, ry = 0, 0
    while k:
        if k & 1:
            if rx == 0 and ry == 0:
                rx, ry = x, y
            else:
                rx, ry = ec_add_fast(rx, ry, x, y, a, p)
        x, y = ec_add_fast(x, y, x, y, a, p)
        k >>= 1
    return rx, ry

@njit
def sm2_sign_fast(msg: int, d: int, Gx: int, Gy: int, n: int, a: int, p: int):
    e = msg
    while True:
        k = random.randint(1, n - 1)
        x1, y1 = ec_mul_fast(k, Gx, Gy, a, p)
        r = (e + x1) % n
        if r == 0 or r + k == n:
            continue
        s = (pow(1 + d, -1, n) * (k - r * d)) % n
        if s != 0:
            return r, s

# ✅ 测试运行与对比
def run_test():
    d = random.randint(1, int(n) - 1)
    msg = random.getrandbits(256)

    # 原始实现耗时
    t0 = time.time()
    r1, s1 = sm2_sign(msg, d)
    t1 = time.time()

    # Numba 加速实现耗时（使用 int 类型）
    t2 = time.time()
    r2, s2 = sm2_sign_fast(msg, d, int(Gx), int(Gy), int(n), int(a), int(p))
    t3 = time.time()

    print(f"✅ 原始签名: (r={r1}, s={s1})")
    print(f"⚡ Numba签名: (r={r2}, s={s2})")
    print(f"🕰️ 原始实现耗时: {(t1 - t0):.6f} 秒")
    print(f"⚡ Numba 加速耗时: {(t3 - t2):.6f} 秒")

run_test()


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at /tmp/ipykernel_36/3659232310.py (78)[0m
[1m
File "../../tmp/ipykernel_36/3659232310.py", line 78:[0m
[1m<source missing, REPL/exec in use?>[0m 

This error may have been caused by the following argument(s):
- argument 0: Int value is too large: 110506044917781079945701285771247294490797758783582682264048967431516402188792
- argument 1: Int value is too large: 92723367749504484986619352596264674043199249626167270835966300609266529524133
- argument 2: Int value is too large: 22963146547237050559479531362550074578802567295341616970375194840604139615431
- argument 3: Int value is too large: 85132369209828568825618990617112496413088388631904505083283536607588877201568
- argument 4: Int value is too large: 115792089210356248756420345214020892766061623724957744567843809356293439045923
- argument 5: Int value is too large: 11579208921035624875